# 生物先端コース 学生実習 2025後期

## 栽培植物起原学分野担当・課題レポート

本実習3日目に実施したイネいもち病菌のRNA-seq解析について、下記の解析を行いレポートを書いてください。

レポートはword等で作成し、Pandaから提出してください。期限は明日から1週間(10/15)です。

### 課題概要

イネいもち病菌株GZをオオムギ品種Nigrateに接種し、0h後・24h後・48h後にそれぞれRNAを抽出し、NextSeq 500シーケンサーを用いてRNA-seqを実施した。

(0時間後を非感染時、48時間後を感染時のデータとして実習では用いた。)

イネいもち病菌はオオムギ品種Nigrateに感染することが知られている。そのため、本解析によって得られる結果から、イネいもち病菌が植物に感染するためにどの様な遺伝子の発現が必要なのかを調べることが出来ると考えられる。

下に示すコマンドとプログラムを実行することで、

* 接種後0h vs 接種後48hのDEG解析 (実習で実施した解析)

* 接種後0h vs 接種後24hのDEG解析

を行う事が出来る。

得られた結果を基に、発現量に大きく変動があった遺伝子についてBLAST検索などを行い、イネいもち病菌が感染する際に重要と考えられる遺伝子について考察を加えてレポートを作成してください。

少なくとも実習で実施した"接種後0h vs 接種後48hのDEG解析"の結果については必ず述べてください。

"接種後0h vs 接種後24hのDEG解析"の結果については、レポートに加えてあった場合は加点対象とします。(無くても合格点は与える。)

また、この頁の最後にデータのURL一覧を載せているので、他に解析を思いついた方は独自に加えて頂いてもかまいません。




## RNA-seqリードのマッピング・リードカウント

下記コマンドで`RNA_seq_0_24_48.sh`というシェルスクリプトをダウンロードし、実行すると、

* 各サンプルのBAMファイル`XXX.bam`
* 全データに基づく遺伝子のアノテーション(GTF)ファイル`transcripts.gtf`
* 各遺伝子の配列`transcripts.fasta`
* リードカウントのファイル`counts.txt`

が作成される。(少し時間がかかる。)

```
wget https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/RNA_seq_0_24_48.sh

bash RNA_seq_0_24_48.sh
```
ちなみに、`RNA_seq_0_24_48.sh`には下の様なコマンドが書かれていて、これらを一気に実施している。

```
# 使用するプログラムのインストール
apt install hisat2
apt install seqkit
apt install samtools
apt install stringtie
apt install subread
apt install gffread
pip install pydeseq2

# 初日作成したリファレンスゲノムのダウンロード
wget https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/assembly.fasta

# マッピング用のindex作成
hisat2-build assembly.fasta assembly

# 各条件・反復ごとにシーケンスデータをダウンロードしマッピングを実施
for i in infect_0h_rep1 infect_0h_rep2 infect_0h_rep3 infect_24h_rep1 infect_24h_rep2 infect_24h_rep3 infect_48h_rep1 infect_48h_rep2 infect_48h_rep3;do
    wget -O ${i}.fastq.gz https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/${i}.fastq.gz
    hisat2 -x assembly -U ${i}.fastq.gz | samtools sort -O bam > ${i}.bam
    samtools index ${i}.bam
done

# 遺伝子アノテーション, `*`はワイルドカードで任意の文字列という意味。
stringtie *.bam -o transcripts.gtf

# 遺伝子毎にリード数カウント
featureCounts -a transcripts.gtf -o counts.txt *.bam

# 各遺伝子の配列を取得
gffread transcripts.gtf -g assembly.fasta -w transcripts.fasta
```




## DEG解析 (0h vs 48h)

実習で実施したDEG解析のプログラムを下に示す。

実行すると`DEG_0h_vs_48h.csv`という接種後0h vs 48hのDEG解析の結果が作成される。

`padj`や`log2FoldChange`に基づき重要そうな遺伝子をピックアップし、[BLAST](https://blast.ncbi.nlm.nih.gov/Blast.cgi)などを使用し機能予測してみてください。

In [ ]:
import pandas as pd
from pydeseq2.utils import load_example_data
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

# データの整備
counts_df = pd.read_csv("counts.txt", sep="\t", comment="#").set_index("Geneid").iloc[:, [5,6,7,11,12,13]].T
metadata = pd.DataFrame(index=counts_df.index, data={"condition":["0h", "0h", "0h", "48h", "48h", "48h"]})

# 発現量の極端に少ない遺伝子を除去
genes_to_keep = counts_df.columns[counts_df.sum(axis=0) >= 10]
counts_df = counts_df[genes_to_keep]

# リードカウントの補正&検定など
inference = DefaultInference()
dds = DeseqDataSet(
  counts=counts_df,
  metadata=metadata,
  design_factors="condition",
  refit_cooks=True,
  inference=inference,
)
dds.deseq2()
stat_res_0_vs_48 = DeseqStats(dds, contrast=["condition", "48h", "0h"], inference=inference)
stat_res_0_vs_48.summary()
stat_res_0_vs_48.results_df.to_csv("DEG_0h_vs_48h.csv")

興味のある遺伝子の配列は下記コマンドで表示させる事が出来ます。

※遺伝子名の後ろに".1"をつけてください。

例) `"STRG.123"`を探す場合は`"STRG.123.1"`

```
seqkit grep -p "探したい遺伝子名" transcripts.fasta > 出力ファイル名
```

## DEG解析 (0h vs 24h)

続いて、下記プログラムを実行すると`DEG_0h_vs_24h.csv`という接種後0h vs 24hのDEG解析の結果が作成される。

レポートにこの結果を加えるかどうかは任意ですが、加点が欲しい場合はこちらの結果についても考察で触れてください。

In [ ]:
import pandas as pd
from pydeseq2.utils import load_example_data
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

# データの整備
counts_df = pd.read_csv("counts.txt", sep="\t", comment="#").set_index("Geneid").iloc[:, [5,6,7,8,9,10]].T
metadata = pd.DataFrame(index=counts_df.index, data={"condition":["0h", "0h", "0h", "24h", "24h", "24h"]})

# 発現量の極端に少ない遺伝子を除去
genes_to_keep = counts_df.columns[counts_df.sum(axis=0) >= 10]
counts_df = counts_df[genes_to_keep]

# リードカウントの補正&検定など
inference = DefaultInference()
dds = DeseqDataSet(
  counts=counts_df,
  metadata=metadata,
  design_factors="condition",
  refit_cooks=True,
  inference=inference,
)
dds.deseq2()
stat_res_0_vs_24 = DeseqStats(dds, contrast=["condition", "24h", "0h"], inference=inference)
stat_res_0_vs_24.summary()
stat_res_0_vs_24.results_df.to_csv("DEG_0h_vs_24h.csv")

# データ解析についての手法の書き方

解析の手法をレポートなどに記述する際には、実施した解析の内容とそれを実施したプログラム・ソフトウェア名を記述します。

実際のコマンドなどは不要です。

例)

XXXのサンプルから抽出したRNAについて、NextSeq 500シーケンサーを用いてシーケンスを実施した。得られたシーケンスデータを使用し、XXXを用いて作成したリファレンスゲノムに対しマッピングを行った。マッピングは[ソフトウェア名]を用いて実施した。...etc

論文では再現性を担保するために使用したソフトウェアのバージョンや、オプションなどの設定も記述しますが、今回は記述しなくて大丈夫です。

# シーケンスデータの場所

自分で全ての解析をやりたい・独自の解析をやりたい方に向けたデータURL一覧

実習1日目に作成したリファレンスゲノム
* https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/assembly.fasta

各時間(0, 24h, 48h)条件でのRNA-seqリード
* 接種して0h後の３反復
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_0h_rep1.fastq.gz
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_0h_rep2.fastq.gz
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_0h_rep3.fastq.gz

* 接種して24h後の３反復
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_24h_rep1.fastq.gz
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_24h_rep2.fastq.gz
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_24h_rep3.fastq.gz

* 接種して48h後の３反復
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_48h_rep1.fastq.gz
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_48h_rep2.fastq.gz
  * https://raw.githubusercontent.com/slt666666/MOryzae_testdata/refs/heads/main/infect_48h_rep3.fastq.gz